In [1]:
import xarray as xr

fn = r'D:\bus2\sst.mnmean.nc'
ds = xr.open_dataset(fn)


In [7]:
import numpy as np

region = ds['sst'].sel(lat=slice(5, -5), lon=slice(190, 240))
lat = region['lat']
weights = np.cos(np.deg2rad(region['lat']))

area_weighted_mean = region.weighted(weights).mean(dim=('lat', 'lon'))

In [8]:
import pandas as pd
var_time = ds['time']
years = pd.to_datetime(var_time).year
months = pd.to_datetime(var_time).month

df = pd.DataFrame({
    'year': years,
    'month': months,
    'sst': area_weighted_mean})
df['sst_running'] = df['sst'].rolling(window=3, center=True).mean()


In [35]:
target_syears = [1950]
dfs = []
for target_syear in target_syears:
    clim_syear = target_syear - 14 
    clim_eyear = target_syear + 15
    df_clim = df[['month', 'sst_running']][(df['year'] >= clim_syear) & (df['year'] <= clim_eyear)].groupby('month').mean().reset_index()
    df_clim.columns = ['month', 'sst_clim']
    for year in range(target_syear, target_syear+5+1):
        df_merged = pd.merge(df[df['year'] == year], df_clim, on='month', how='left')
        dfs.append(df_merged)
df_oni = pd.concat(dfs)
df_oni['diff'] = df_oni['sst_running'] - df_oni['sst_clim']


In [40]:
for i in range(0, len(df_oni['diff']), 12):
    group = df_oni['diff'].iloc[i:i+12]
    rounded = [f"{x:.3f}" for x in group]  # 소수점 둘째자리 반올림
    print(','.join(rounded))

-1.509,-1.336,-1.164,-1.181,-1.072,-0.851,-0.535,-0.425,-0.385,-0.444,-0.601,-0.821
-0.802,-0.541,-0.166,0.176,0.360,0.584,0.700,0.887,0.993,1.153,1.041,0.790
0.555,0.366,0.340,0.295,0.205,0.001,-0.078,0.002,0.151,0.104,0.044,0.129
0.423,0.599,0.625,0.659,0.752,0.773,0.746,0.729,0.782,0.844,0.836,0.795
0.778,0.468,-0.046,-0.410,-0.538,-0.496,-0.641,-0.844,-0.899,-0.770,-0.731,-0.676
-0.659,-0.615,-0.690,-0.803,-0.787,-0.720,-0.683,-0.749,-1.087,-1.425,-1.671,-1.436
